### Run web-hosted BOPTEST simulation 

For running BOPTEST without the need of local downloads or installations, this simulation uses the BOPTEST  web-hosted environment (BOPTEST-Service). This publicly accessible service allows for the deployment of the test cases in the cloud, enabling interactions via the BOPTEST API. This notebook follows the steps outlined in [this](https://github.com/ibpsa/project1-boptest/blob/master/docs/workshops/BS21Workshop_20210831/Introduction_to_the_BOPTEST_framework.ipynb) BOPTEST tutorial.  

Prior to initiating the simulation, begin by importing the interface for the control application and selecting the control application, test case and simulation parameters.

### Selecting a  Control, Test Case and Simulation Parameters
We can select and launch a test case from the repository of [currently available test cases](https://ibpsa.github.io/project1-boptest/testcases/index.html). The simulation paramenters such as the scenario ones (time period and electricity price) can be chosen from the [test cases directory](https://github.com/ibpsa/project1-boptest/tree/master/testcases). Here we can also define the simulation step in seconds. 

In [1]:
import os
import sys
sys.path.append("..")

# Change BOPTEST interface!!!
from BOPTest_interface_os_c_1a_zone_temp_adjs_rat import (
    BOPTestControls,
    BOPTestControlFunctions,
    BOPTestInterface
)

# Select control and test case identifier 
control = 'os_c_1a_zone_temp_adjs_rat' 

# os_c_1a_zone_temp_adj_inc_ratch
# os_c_1b_zone_temp_adj_perform_ratch
# os_c_2a_zone_pre_heat_cool_sim
# os_c_2b_zone_pre_heat_cool_com
# os_c_1a2a_zone_temp_adj_inc_ratch_pre_heat_cool_sim
# os_c_1a2b_zone_temp_adj_inc_ratch_pre_heat_cool_com
# os_c_1b2a_zone_temp_adj_perform_ratch_pre_heat_cool_sim
# os_c_1b2b_zone_temp_adj_perform_ratch_pre_heat_cool_com
# os_c_3_chi_water_temp_reset

testcase = 'bestest_air' # bestest_air, bestest_hydronic_heat_pump, singlezone_commercial_hydronic, multizone_office_simple_air
graph_path = os.path.join(os.getcwd(), f'testcase_{testcase}', f'{testcase}.ttl')

# Select the simulation parameters/scenario (choosen from https://github.com/ibpsa/project1-boptest/tree/master/testcases)
time_period = 'peak_heat_day'
electricity_price ='highly_dynamic'

electricity_price_identifier = 'PriceElectricPowerHighlyDynamic'

# Set simulation step 
step = 900 # 15 min


### Create Application 
Create application based on the selected control. 

In [2]:
from utility_functions import get_hvac_signals

from utility_functions.get_price_threshold import get_price_threshold

class TestCaseInterface(BOPTestInterface):
    def __init__(self, controls, config_path, hvac_signal_function):
        super().__init__(controls, config_path)
        self.hvac_signal_function = hvac_signal_function

    def get_hvac_signals(self, *args):
        return self.hvac_signal_function(*args)
    
    def get_price_threshold(self, *args):
        return get_price_threshold(*args)
    
def create_application(config_path, hvac_signal_function_name) -> BOPTestInterface:
    '''takes a config file and returns an instantiated TestCaseInterface
    
    Parameters
    ----------
        config_path : str
            the path to a configuration file for instantiating a bestest air shed application 
    Returns
    -------
        application : TestCaseInterface
            returns an instantiated bestest air interface with shedding controls
'''
    
    functions = BOPTestControlFunctions()
    controls = BOPTestControls(functions)
    hvac_signal_function = getattr(get_hvac_signals, hvac_signal_function_name)
    
    # Instantiate the TestCaseInterface with the controls, config_path, and hvac_signal_function
    application = TestCaseInterface(controls, config_path, hvac_signal_function)
    return application


### Instantiate Application 
Instantiate the test case interface with the selected control.

In [3]:
import os
config_path = os.path.join(os.getcwd(), f'testcase_{testcase}', f'config_{control}.yml')
function_name = 'get_' + testcase + '_hvac_signals'

bt_interface = create_application(config_path, function_name)

# print of the control's SPARQL result

(rdflib.term.URIRef('urn:bldg/hvacZone'), rdflib.term.Literal('zon_reaTRooAir_y'), rdflib.term.Literal('LowerSetp[1]'), rdflib.term.Literal('UpperSetp[1]'), rdflib.term.Literal('Occupancy[1]'), rdflib.term.Literal('con_oveTSetHea_u'), rdflib.term.Literal('con_oveTSetCoo_u'), None, None, None, None, None)


### Launch the Selected Test Case
First, import the Python `requests` library so that we can make HTTP requests to the BOPTEST API at the address indicated by the `url`. Then, use the `POST /testcases/<test_case_name>/select` BOPTEST API endpoint to launch the test case and receive a corresponding `testid`. While the url is the common gateway for everyone to access the BOPTEST web-service, the `testid` is a unique identifier for you to address the test case that you have selected and launched.

In [4]:
import requests
url = "https://api.boptest.net"
testid = \
    requests.post('{0}/testcases/{1}/select'.format(url,testcase)).json()['testid']


Once we have successfully obtained the `testid`, it is possible to start interacting with the selected test case using the rest of the BOPTEST API. For example, use the `GET /name` BOPTEST API endpoint, along with the testid, to request the name of the chosen test case.

In [5]:
# Get test case name
name = requests.get('{0}/name/{1}'.format(url, testid)).json()['payload']
print(name)

{'name': 'bestest_air'}


### Getting Control Input and Measurement Points
While control input and measurement points are described in the [documentation](https://ibpsa.github.io/project1-boptest/testcases/), they are also available to retreive from the BOPTEST API. This is especially useful to store for later when requesting data for a specific point.

Retrieve the control input and measurement outputs using the `GET /inputs` and `GET /measurements` BOPTEST API endpoints.

In [6]:
# Get inputs available
inputs = requests.get('{0}/inputs/{1}'.format(url, testid)).json()['payload']
print('TEST CASE INPUTS ---------------------------------------------')
print(inputs.keys())
# Get measurements available
measurements = requests.get('{0}/measurements/{1}'.format(url, testid)).json()['payload']
print('TEST CASE MEASUREMENTS ---------------------------------------')
print(measurements.keys())

TEST CASE INPUTS ---------------------------------------------
dict_keys(['con_oveTSetCoo_activate', 'con_oveTSetCoo_u', 'con_oveTSetHea_activate', 'con_oveTSetHea_u', 'fcu_oveFan_activate', 'fcu_oveFan_u', 'fcu_oveTSup_activate', 'fcu_oveTSup_u'])
TEST CASE MEASUREMENTS ---------------------------------------
dict_keys(['fcu_reaFloSup_y', 'fcu_reaPCoo_y', 'fcu_reaPFan_y', 'fcu_reaPHea_y', 'zon_reaCO2RooAir_y', 'zon_reaPLig_y', 'zon_reaPPlu_y', 'zon_reaTRooAir_y', 'zon_weaSta_reaWeaCeiHei_y', 'zon_weaSta_reaWeaCloTim_y', 'zon_weaSta_reaWeaHDifHor_y', 'zon_weaSta_reaWeaHDirNor_y', 'zon_weaSta_reaWeaHGloHor_y', 'zon_weaSta_reaWeaHHorIR_y', 'zon_weaSta_reaWeaLat_y', 'zon_weaSta_reaWeaLon_y', 'zon_weaSta_reaWeaNOpa_y', 'zon_weaSta_reaWeaNTot_y', 'zon_weaSta_reaWeaPAtm_y', 'zon_weaSta_reaWeaRelHum_y', 'zon_weaSta_reaWeaSolAlt_y', 'zon_weaSta_reaWeaSolDec_y', 'zon_weaSta_reaWeaSolHouAng_y', 'zon_weaSta_reaWeaSolTim_y', 'zon_weaSta_reaWeaSolZen_y', 'zon_weaSta_reaWeaTBlaSky_y', 'zon_weaSta_re

### Initializing a test scenario
Select a testing scenario using the `PUT /scenario` BOPTEST API endpoint to allow to test the  controller for a pre-defined period and with predefined conditions. It uses the parameters time_period and electricity_price to set a specific scenario as defined for each test case within their respective [documentation](https://github.com/ibpsa/project1-boptest/tree/master/testcases). 

In [7]:
scenario_return = requests.put('{0}/scenario/{1}'.format(url, testid),
    data={'time_period':time_period,
    'electricity_price':electricity_price}).json()['payload']

Setting the scenario API call can return the `time_period` as an argument with current time measurements of the simulation intialized to the scenario time period.
The variable `start_time_days` refers to the start time of the test period.

In [8]:
y = scenario_return['time_period']
start_time_days = y['time']/24/3600

###  Get Forecast Data for Plotting  BOPTEST comfort range

Use the `PUT /forecast` BOPTEST API endpoint with parameters point names referring to the list of forecast data needed, as well as horizon and interval in seconds to set a forecast horizon and data interval. This same endpoint is used to retrieve the forecast data, which will start from the current simulation time.

In [9]:
import pandas as pd
df_f_single_day = pd.DataFrame()
forecast_points = requests.get('{0}/forecast_points/{1}'.format(url, testid)).json()['payload']
list_forecast = list(forecast_points.keys())

# Get the forecast data for the specified point names and time horizon
f = requests.put('{0}/forecast/{1}'.format(url, testid), 
             data={'point_names': list_forecast, 'horizon':14*24*3600, 'interval':1*step}).json()['payload']
# Create a dataframe from the forecast data
df_f_single_day = pd.DataFrame.from_dict(f)
    
# Set the index of the dataframe to the time field
df_f_single_day.set_index('time', inplace=True)
    
# Print the dataframe
#print(df_f_single_day)

### Simulating the Baseline Control for Benchmarking
Before testing a new controller, we need to simulate the baseline for a defined scenario and evaluate its performance. The baseline can be quickly simulated by setting a long control step and without overwritting any control signal when advancing the simulation. Here, we even set the control step to one year knowing that the simulation is forced to stop at the end of the scenario time period.

In [10]:
# Set the control step to one year 
requests.put('{0}/step/{1}'.format(url, testid), data={'step':365*24*3600})

# Advance the simulation with no control overwrite
requests.post('{0}/advance/{1}'.format(url, testid), data={}).json()

{'status': 200,
 'message': 'Advanced simulation successfully from 28857600.0s to 30067200.0s.',
 'payload': {'time': 30067200,
  'fcu_reaFloSup_y': -1.5236703275459965e-212,
  'fcu_reaPCoo_y': 4.304578790366723e-209,
  'fcu_reaPFan_y': 0,
  'fcu_reaPHea_y': 0,
  'zon_reaCO2RooAir_y': 426.1319838078747,
  'zon_reaPLig_y': 56.64000000000001,
  'zon_reaPPlu_y': 25.92,
  'zon_reaTRooAir_y': 289.9550854762039,
  'zon_weaSta_reaWeaCeiHei_y': 77777,
  'zon_weaSta_reaWeaCloTim_y': 30067200,
  'zon_weaSta_reaWeaHDifHor_y': 0,
  'zon_weaSta_reaWeaHDirNor_y': 0,
  'zon_weaSta_reaWeaHGloHor_y': 0,
  'zon_weaSta_reaWeaHHorIR_y': 216,
  'zon_weaSta_reaWeaLat_y': 0.6939429105929453,
  'zon_weaSta_reaWeaLon_y': -1.830152253641254,
  'zon_weaSta_reaWeaNOpa_y': 0,
  'zon_weaSta_reaWeaNTot_y': 0,
  'zon_weaSta_reaWeaPAtm_y': 101325,
  'zon_weaSta_reaWeaRelHum_y': 0.8200000000000001,
  'zon_weaSta_reaWeaSolAlt_y': -1.2823087938201239,
  'zon_weaSta_reaWeaSolDec_y': -0.4059129605677608,
  'zon_weaSta_reaW

### Getting Core KPI Values
To obtain the KPI, calculated based on the emulator performance during a simulation (starting from the initialization), use the `GET /kpi` BOPTEST API endpoint.

The core KPIs, those that are calculated for every test case, are defined in BOPTEST to evaluate different aspects of a controller. These are:

- **Thermal discomfort**: reported with units of [Kh/zone], defines the cumulative deviation of zone temperatures from upper and lower comfort limits that are predefined within the test case FMU for each zone, averaged over all zones. Air temperature is used for air-based systems and operative temperature is used for radiant systems.

- **Indoor Air Quality (IAQ) Discomfort**: reported with units of [ppmh/zone], defines the extent that the CO
 concentration levels in zones exceed bounds of the acceptable concentration level, which are predefined within the test case FMU for each zone, averaged over all zones.

- **Energy Use**: reported with units of [kWh/m2], defines the HVAC energy usage.

- **Peak electrical demand**:reported with units of [kW/m2], defines the HVAC peak electrical demand.

- **Peak gas demand**:reported with units of [kW/m2], defines the HVAC peak gas demand.

- **Peak district heating demand**:reported with units of [kW/m2], defines the HVAC peak district heating demand.

- **Cost**: reported with units of [USD/m2] or [EUR/m2], defines the operational cost associated with the HVAC energy usage.

- **Emissions**: reported with units of [kgCO2/m2], defines the CO2 emissions from the HVAC energy usage.

- **Computational time ratio**: defines the average ratio between the controller computation time and the test simulation control step. The controller computation time is measured as the time between two emulator advances.

The warmup period is not taken into account for the calculation of the KPIs.


In [11]:
kpis_bas = requests.get('{0}/kpi/{1}'.format(url, testid)).json()['payload']
kpis_bas

{'tdis_tot': 5.687288484421467,
 'idis_tot': 1221.6529527806554,
 'ener_tot': 3.7227262062358055,
 'cost_tot': 0.0157791922893377,
 'emis_tot': 0.785590772991245,
 'pele_tot': 0.010216143907980911,
 'pgas_tot': 0.12133848299596474,
 'pdih_tot': None,
 'time_rat': None}

### Query graph

In [12]:
import rdflib
g = rdflib.Graph()
g.parse(graph_path)

query = f"""
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
SELECT ?Epower_point ?Tpower_point WHERE {{
    OPTIONAL {{
    ?Epoint a brick:Electric_Power_Sensor ;
        ref:hasExternalReference/ref:hasTimeseriesId ?Epower_point .}}

    OPTIONAL {{
    ?Tpoint a brick:Thermal_Power_Sensor ;
        ref:hasExternalReference/ref:hasTimeseriesId ?Tpower_point .}}    

}}
"""
list_power_measurement = []
# Execute the query and process the results
for row in g.query(query):
    Epower_point = row[0]
    Tpower_point = row[1]
    list_power_measurement.append(Epower_point.value)
    if Tpower_point:
        list_power_measurement.append(Tpower_point.value)
    # Do something with power_point, e.g., print it or add to a list
print(list_power_measurement)

query = f"""
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
SELECT ?zone_set_temp_heat_point ?zone_set_temp_cool_point ?zone_set_temp_point ?zone_temp_point WHERE {{
    
    ?zone a brick:Zone ;
    brick:hasPoint ?TZon, ?TSetHeaZon, ?TSetCooZon, ?TSetZon .
    
    ?TZon a brick:Zone_Air_Temperature_Sensor ;
        ref:hasExternalReference/ref:hasTimeseriesId ?zone_temp_point .
    
    OPTIONAL {{
    ?TSetHeaZon a brick:Zone_Air_Heating_Temperature_Setpoint;
        ref:hasExternalReference/ref:hasTimeseriesId ?zone_set_temp_heat_point .}}

    OPTIONAL {{
    ?TSetCooZon a brick:Zone_Air_Cooling_Temperature_Setpoint;
        ref:hasExternalReference/ref:hasTimeseriesId ?zone_set_temp_cool_point .}}

    OPTIONAL {{
    ?TSetZon a brick:Zone_Air_Temperature_Setpoint;
        ref:hasExternalReference/ref:hasTimeseriesId ?zone_set_temp_point .}}  

}} GROUP BY ?zone
"""
list_temp_measurement = []
# Execute the query and process the results
for row in g.query(query):
    zone_set_temp_heat_point = row[0]
    zone_set_temp_cool_point = row[1]
    zone_set_temp_point = row[2]
    zone_temp_point = row[3]
    if zone_set_temp_heat_point != None:
       list_temp_measurement.append(zone_set_temp_heat_point.value)
    if zone_set_temp_cool_point != None:
        list_temp_measurement.append(zone_set_temp_cool_point.value)
    if zone_set_temp_point != None:
        list_temp_measurement.append(zone_set_temp_point.value)
    if zone_temp_point != None:
        list_temp_measurement.append(zone_temp_point.value)
    # Do something with power_point, e.g., print it or add to a list
print(list_temp_measurement)

list_all_measurement = list_power_measurement + list_temp_measurement


query = f"""
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
SELECT ?set_temp_min_point ?set_temp_max_point ?occ_sensor_point WHERE {{
    
    ?zone a brick:Zone ;
    brick:hasPoint ?TSetMin, ?TSetMax, ?Occ .
    
    ?TSetMin a brick:Min_Air_Temperature_Setpoint ;
        ref:hasExternalReference/ref:hasTimeseriesId ?set_temp_min_point .

    ?TSetMax a brick:Max_Air_Temperature_Setpoint ;
        ref:hasExternalReference/ref:hasTimeseriesId ?set_temp_max_point .

    ?Occ a brick:Occupancy_Sensor ;
        ref:hasExternalReference/ref:hasTimeseriesId ?occ_sensor_point .

}} GROUP BY ?zone
"""
list_forecast = []
# Execute the query and process the results
for row in g.query(query):
    set_temp_min_point = row[0]
    set_temp_max_point = row[1]
    occ_sensor_point = row[2]
    if set_temp_min_point != None:
        list_forecast.append(set_temp_min_point.value)
    if set_temp_max_point != None:
        list_forecast.append(set_temp_max_point.value)
    if occ_sensor_point != None:
        list_forecast.append(occ_sensor_point.value)
    # Do something with power_point, e.g., print it or add to a list
print(list_forecast)


['fcu_reaPFan_y', 'fcu_reaPHea_y', 'fcu_reaPCoo_y', 'fcu_reaPHea_y']
['con_oveTSetHea_u', 'con_oveTSetCoo_u', 'zon_reaTRooAir_y']
['LowerSetp[1]', 'UpperSetp[1]', 'Occupancy[1]']


### Export Baseline Results

In [13]:
# Export baseline results
import numpy as np


df_pow = pd.DataFrame()
for point in list_power_measurement:
        args = {'point_names':point,
                'start_time':start_time_days*24*3600, 
                'final_time':np.inf}
        pow = requests.put('{0}/results/{1}'.format(url, testid), 
                           data=args).json()['payload']
        df_pow = pd.concat((df_pow,pd.DataFrame(data=pow[point], 
                            index=pow['time'],columns=[point])), axis=1)
df_pow.index.name = 'time'

# Export DataFrame to CSV file

csv_filepower_path = os.path.join(os.getcwd(), f'testcase_{testcase}', f'results_{control}', 'default_scenario','baseline_powerData.csv')
df_pow.to_csv(csv_filepower_path)

csv_fileschedule_path = os.path.join(os.getcwd(), f'testcase_{testcase}', f'results_{control}', 'default_scenario','scheduleData.csv')
df_f_single_day.to_csv(csv_fileschedule_path)



df_temp = pd.DataFrame()
for tpoint in list_temp_measurement:
        args = {'point_names':tpoint,
                'start_time':start_time_days*24*3600, 
                'final_time':np.inf}
        temp = requests.put('{0}/results/{1}'.format(url, testid), 
                           data=args).json()['payload']
        df_temp = pd.concat((df_temp,pd.DataFrame(data=temp[tpoint], 
                            index=temp['time'],columns=[tpoint])), axis=1)
df_temp.index.name = 'time'

# Export DataFrame to CSV file
csv_fileTemp_path = os.path.join(os.getcwd(), f'testcase_{testcase}', f'results_{control}', 'default_scenario','baseline_tempData.csv')
df_temp.to_csv(csv_fileTemp_path)


df_all = pd.DataFrame()
for tpoint in list_all_measurement:
        args = {'point_names':tpoint,
                'start_time':start_time_days*24*3600, 
                'final_time':np.inf}
        all = requests.put('{0}/results/{1}'.format(url, testid), 
                           data=args).json()['payload']
        df_all = pd.concat((df_all,pd.DataFrame(data=all[tpoint], 
                            index=all['time'],columns=[tpoint])), axis=1)
df_all.index.name = 'time'

# Export DataFrame to CSV file
csv_fileTemp_path = os.path.join(os.getcwd(), f'testcase_{testcase}', f'results_{control}', 'default_scenario','baseline.csv')
df_all.to_csv(csv_fileTemp_path)


print("Baseline data has been exported to data.csv successfully.")


Baseline data has been exported to data.csv successfully.


### Assessing a New Control Strategy
For assessing a new control strategy, we need to initialize the test case in the same scenario used to obtain the baseline controller KPIs, and here we obtain the control step as defined in the setting simulation parameters part. After that, we instantiate the controller and define a simulation loop using the BOPTEST API. Note that the test case will return an empty dictionary when the testing period has elapsed, such that we can loop over the returned measurements without having to infer when the simulation time should end.

In [14]:
import json

# Initialize scenario
y = requests.put('{0}/scenario/{1}'.format(url, testid), 
                 data={'time_period': time_period,
                       'electricity_price':electricity_price}).json()['payload']['time_period']

# Set control step
requests.put('{0}/step/{1}'.format(url, testid), data={'step':step})

# Get step
s = requests.get('{0}/step/{1}'.format(url, testid)).json()['payload']

# Instantiate controller
con = bt_interface

f_price = requests.put('{0}/forecast/{1}'.format(url, testid), 
                    data={'point_names': [electricity_price_identifier], 'horizon':14*24*3600, 'interval':1*3600}).json()['payload'] 
    

# Simulation loop
from IPython.display import clear_output
while y:

    # Clear the display output at each step
    clear_output(wait=True)

    current_time = y['time']

    # Print the current operative temperature and simulation time
    simulation_time_days = y['time']/3600/24
    print('Simulation time [elapsed days] = {:.2f}'.format((simulation_time_days - \
                                                    start_time_days)))
    print('-------------------------------------------------------------------')

    # Get forecast
    # points = requests.get('{0}/forecast_points/{1}'.format(url, testid))

    # # Parse the response content into a dictionary
    # point_data = json.loads(points.content)['payload']

    # # Extract the point names from the dictionary
    # point_names = [p for p in point_data.keys()]

    point_names = [electricity_price_identifier] + list_forecast

    # Use the point names as input for the PUT /forecast endpoint
    f = requests.put('{0}/forecast/{1}'.format(url, testid), 
                    data={'point_names': point_names, 'horizon':24*3600, 'interval':1*step}).json()['payload'] 
    
    # Compute control signal (measurements, forecast, step)
    u = con.control_agent(y, f, s, start_time_days, current_time, f_price)

    # Advance simulation with control signal
    y = requests.post('{0}/advance/{1}'.format(url, testid), data=u).json()['payload']    


Simulation time [elapsed days] = 14.00
-------------------------------------------------------------------
30067200 900
{0: 0}
price_threshold_value 0.0378966
range(0, 1) ['urn:bldg/hvacZone'] ['zon_reaTRooAir_y'] [] ['con_oveTSetHea_u'] ['con_oveTSetCoo_u'] ['LowerSetp[1]'] ['UpperSetp[1]'] ['Occupancy[1]'] [] [] [] None None None
heat
TSetHeaZon 288.15 TSetHeaZon_baseline [288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15, 288.15

### Export Flexible Results

In [15]:
# Export DF results
import numpy as np


df_pow = pd.DataFrame()
for point in list_power_measurement:
        args = {'point_names':point,
                'start_time':start_time_days*24*3600, 
                'final_time':np.inf}
        pow = requests.put('{0}/results/{1}'.format(url, testid), 
                           data=args).json()['payload']
        df_pow = pd.concat((df_pow,pd.DataFrame(data=pow[point], 
                            index=pow['time'],columns=[point])), axis=1)
df_pow.index.name = 'time'

# Export DataFrame to CSV file
csv_filepower_path = os.path.join(os.getcwd(), f'testcase_{testcase}', f'results_{control}', 'default_scenario','flexible_powerData.csv')
df_pow.to_csv(csv_filepower_path)



df_temp = pd.DataFrame()
for tpoint in list_temp_measurement:
        args = {'point_names':tpoint,
                'start_time':start_time_days*24*3600, 
                'final_time':np.inf}
        temp = requests.put('{0}/results/{1}'.format(url, testid), 
                           data=args).json()['payload']
        df_temp = pd.concat((df_temp,pd.DataFrame(data=temp[tpoint], 
                            index=temp['time'],columns=[tpoint])), axis=1)
df_temp.index.name = 'time'

# Export DataFrame to CSV file
csv_fileTemp_path = os.path.join(os.getcwd(), f'testcase_{testcase}', f'results_{control}', 'default_scenario','flexible_tempData.csv')
df_temp.to_csv(csv_fileTemp_path)

print("Flexible data has been exported to data.csv successfully.")

Flexible data has been exported to data.csv successfully.


Get KPIs from Test Control 


In [16]:

# Get and print the test control KPIs
kpis_con = requests.get('{0}/kpi/{1}'.format(url, testid)).json()['payload']
print(kpis_con)

import csv

# Create a list of field names (keys)
field_names = list(kpis_con.keys())

# Open a file in write mode
with open(os.path.join(os.getcwd(), f'testcase_{testcase}', f'results_{control}', 'default_scenario','kpis_con.csv'), 'w') as csvfile:
    # Create a writer object
    writer = csv.DictWriter(csvfile, fieldnames=field_names)
    # Write the header row
    writer.writeheader()
    # Write the dictionary as a row
    writer.writerow(kpis_con)


{'tdis_tot': 79.00139674165719, 'idis_tot': 1220.7152044692407, 'ener_tot': 3.40154959795667, 'cost_tot': 0.014042789506681471, 'emis_tot': 0.7149108975163123, 'pele_tot': 0.009871271000767325, 'pgas_tot': 0.12531074702289055, 'pdih_tot': None, 'time_rat': 0.0019479520548379703}


### Stopping the Test 
We can stop the test case instance in order to free up web-service resources for other test cases. 

In [17]:
# Set to true if want to stop test case
stop = False
if stop:
  requests.put('{0}/stop/{1}'.format(url, testid))
